In [22]:
from confluent_kafka import Consumer, KafkaException, Producer
import json
import numpy as np
import logging
from tensorflow.keras.models import load_model
import os


# Kafka consumer ve producer config
conf = {
    'bootstrap.servers': 'localhost:9092',  # Kafka'nın çalıştığı adres
    'group.id': 'consumer_group',  # Consumer group ID
    'auto.offset.reset': 'earliest'  # İlk veriden başlamak için
}

# Consumer oluştur
consumer = Consumer(conf)
print("Consumer oluşturuldu.")

# Producer için config oluştur
producer = Producer({'bootstrap.servers': 'localhost:9092'})  # Producer config
print("Producer oluşturuldu.")

# Modeli yükle
rf_model = load_model('fraud_detection_lstm_model.h5')
print("Model yüklendi.")

# Kafka topic veri alma
consumer.subscribe(['input_topic'])
print("Topic abonesi olundu.")

import tensorflow as tf

def process_data_and_predict(data):
    data = json.loads(data)
    features = [data['V1'], data['V6'], data['V8'], data['V13'], data['V15'], data['V19'], data['V20'], data['V21'],
                data['V23'], data['V24'], data['V25'], data['V26'], data['V27'], data['V28'], data['log_amount']]
    print(f"Öznitelikler: {features}")
    # Veriyi şekillendir
    features = np.array(features).reshape(1, 1, -1)  # (batch_size, time_steps, features)

    features = tf.convert_to_tensor(features, dtype=tf.float32)  # TensorFlow tensor'una dönüştürülüyor
    prediction = rf_model.predict(features)
    print(f"Model tahmini yapıldı: {prediction[0]}")
    return prediction[0]  # 0: Normal, 1: Anomali



def print_prediction(prediction, msg):
    # msg.key() None olursa kontrol et
    key = str(msg.key()) if msg.key() else None
    
    # Kafka mesajının değerini JSON formatına dönüştürmeden önce decode et
    try:
        value = json.loads(msg.value().decode('utf-8'))  # Byte dizisini JSON'a dönüştür
    except Exception as e:
        print(f"Mesaj decode hatası: {e}")
        return
    
    if prediction == 1:
        print(f"Anomali Tespit Edildi: {value}")
        producer.produce('anomaly_topic', key=key, value=json.dumps(value))  # JSON formatına dönüştür
    else:
        print(f"Normal Veri: {value}")
        producer.produce('normal_topic', key=key, value=json.dumps(value))  # JSON formatına dönüştür
    
    producer.flush()  # Veriyi hemen gönder


# Veriyi al ve işleme
def process_messages():
    while True:
        # Kafka'dan veri alırken timeout süresini artırıyoruz
        msg = consumer.poll(timeout=1.0)  # 1 saniye bekleyerek veri al

        if msg is None:  # Veri yoksa devam et
            print("Topic'te mesaj yok veya timeout süresi doldu.")
            continue
        if msg.error():
            print(f"Hata oluştu: {msg.error()}")
            raise KafkaException(msg.error())

        # Veriyi işleyip tahmin yapma
        prediction = process_data_and_predict(msg.value())

        # Tahmin sonrası sonuçları yazdırma ve Kafka'ya gönderme
        print_prediction(prediction, msg)

# Consumer'ı başlatma
try:
    print("Mesaj işleme başlıyor...")
    process_messages()

except KeyboardInterrupt:
    print("İşlem durduruldu.")
finally:
    consumer.close()
    print("Consumer kapatıldı.")


2024-12-24 15:33:00,923 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


Consumer oluşturuldu.
Producer oluşturuldu.
Model yüklendi.
Topic abonesi olundu.
Mesaj işleme başlıyor...
Öznitelikler: [0.2619373983737463, 0.19231704918773673, 0.03743905332707864, 0.08845212783322454, 0.0037965839167798388, 0.8438398094144369, 0.24533667613939036, 0.26188814794886706, 0.16975361920834564, 0.8938386097094272, 0.1193608248368262, 0.09959310687355039, 0.10054848168130781, 0.48157746255135425, 0.05896181183680038]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Model tahmini yapıldı: [0.9991496]
Normal Veri: {'V1': 0.2619373983737463, 'V6': 0.19231704918773673, 'V8': 0.03743905332707864, 'V13': 0.08845212783322454, 'V15': 0.0037965839167798388, 'V19': 0.8438398094144369, 'V20': 0.24533667613939036, 'V21': 0.26188814794886706, 'V23': 0.16975361920834564, 'V24': 0.8938386097094272, 'V25': 0.1193608248368262, 'V26': 0.09959310687355039, 'V27': 0.10054848168130781, 'V28': 0.48157746255135425, 'log_amount': 0.05896181183680038, 'Class': 0}
Öznitelikler: [0.4740120595462908, 0.436718